In [160]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
pip install pyspark


     |████████████████████████████████| 212.4MB 71kB/s 
     |████████████████████████████████| 204kB 18.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=7406e4d8aa697cdb0468c111b7e1db70848f26298a8e131064d483378b582709
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [161]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Aggriculture').getOrCreate()
df = spark.read.csv('train_yaOffsB.csv', header = True, inferSchema = True)
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Estimated_Insects_Count: integer (nullable = true)
 |-- Crop_Type: integer (nullable = true)
 |-- Soil_Type: integer (nullable = true)
 |-- Pesticide_Use_Category: integer (nullable = true)
 |-- Number_Doses_Week: integer (nullable = true)
 |-- Number_Weeks_Used: integer (nullable = true)
 |-- Number_Weeks_Quit: integer (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Crop_Damage: integer (nullable = true)



In [162]:
df.show(10)

+---------+-----------------------+---------+---------+----------------------+-----------------+-----------------+-----------------+------+-----------+
|       ID|Estimated_Insects_Count|Crop_Type|Soil_Type|Pesticide_Use_Category|Number_Doses_Week|Number_Weeks_Used|Number_Weeks_Quit|Season|Crop_Damage|
+---------+-----------------------+---------+---------+----------------------+-----------------+-----------------+-----------------+------+-----------+
|F00000001|                    188|        1|        0|                     1|                0|                0|                0|     1|          0|
|F00000003|                    209|        1|        0|                     1|                0|                0|                0|     2|          1|
|F00000004|                    257|        1|        0|                     1|                0|                0|                0|     2|          1|
|F00000005|                    257|        1|        1|                     1|          

In [163]:
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
ID,F00000001,F00000003,F00000004,F00000005,F00000006
Estimated_Insects_Count,188,209,257,257,342
Crop_Type,1,1,1,1,1
Soil_Type,0,0,0,1,0
Pesticide_Use_Category,1,1,1,1,1
Number_Doses_Week,0,0,0,0,0
Number_Weeks_Used,0,0,0,0,0
Number_Weeks_Quit,0,0,0,0,0
Season,1,2,2,2,2
Crop_Damage,0,1,1,1,1


In [164]:
df.columns

['ID',
 'Estimated_Insects_Count',
 'Crop_Type',
 'Soil_Type',
 'Pesticide_Use_Category',
 'Number_Doses_Week',
 'Number_Weeks_Used',
 'Number_Weeks_Quit',
 'Season',
 'Crop_Damage']

In [165]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
ID,88858,None,None,F00000001,F00155945
Estimated_Insects_Count,88858,1399.0122104931463,849.0487806668847,150,4097
Crop_Type,88858,0.2843750703369421,0.4511188091519747,0,1
Soil_Type,88858,0.4584167998379437,0.49827064083043343,0,1
Pesticide_Use_Category,88858,2.2641855544801817,0.46177158158329773,1,3
Number_Doses_Week,88858,25.849951608183844,15.554427765396934,0,95
Number_Weeks_Used,79858,28.62397004683313,12.391881462463685,0,67
Number_Weeks_Quit,88858,9.589986270228904,9.900631115271118,0,50
Season,88858,1.8969591933196785,0.701322132743697,1,3


In [166]:
columns=['ID','Estimated_Insects_Count','Crop_Type','Soil_Type','Pesticide_Use_Category','Number_Doses_Week','Number_Weeks_Used',
         'Number_Weeks_Quit','Season','Crop_Damage']

In [167]:
df.toPandas()[columns].nunique()

ID                         88858
Estimated_Insects_Count       71
Crop_Type                      2
Soil_Type                      2
Pesticide_Use_Category         3
Number_Doses_Week             20
Number_Weeks_Used             64
Number_Weeks_Quit             51
Season                         3
Crop_Damage                    3
dtype: int64

In [168]:
numeric_feautres=['Estimated_Insects_Count','Number_Weeks_Used','Number_Weeks_Quit']

categorical_feautres=['Crop_Type','Soil_Type','Pesticide_Use_Category','Number_Doses_Week','Season']

target=['Crop_Damage']

In [169]:
numeric_data = df.select(numeric_feautres).toPandas()

In [170]:
df=df.select('Estimated_Insects_Count','Crop_Type','Soil_Type','Pesticide_Use_Category','Number_Doses_Week','Number_Weeks_Used',
         'Number_Weeks_Quit','Season','Crop_Damage')

cols = df.columns
df.printSchema()

root
 |-- Estimated_Insects_Count: integer (nullable = true)
 |-- Crop_Type: integer (nullable = true)
 |-- Soil_Type: integer (nullable = true)
 |-- Pesticide_Use_Category: integer (nullable = true)
 |-- Number_Doses_Week: integer (nullable = true)
 |-- Number_Weeks_Used: integer (nullable = true)
 |-- Number_Weeks_Quit: integer (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Crop_Damage: integer (nullable = true)



In [171]:
df.toPandas().isna().sum()

Estimated_Insects_Count       0
Crop_Type                     0
Soil_Type                     0
Pesticide_Use_Category        0
Number_Doses_Week             0
Number_Weeks_Used          9000
Number_Weeks_Quit             0
Season                        0
Crop_Damage                   0
dtype: int64

In [172]:
df.filter(df['Number_Weeks_Used'].isNull()).count()

9000

In [173]:
for j in ['Estimated_Insects_Count','Crop_Type','Soil_Type','Pesticide_Use_Category','Number_Doses_Week','Number_Weeks_Used','Number_Weeks_Quit']:
    print("Correlation with"+j+" Crop_Damage",df.corr('Crop_Damage',j))

Correlation withEstimated_Insects_Count Crop_Damage 0.2028883265043235
Correlation withCrop_Type Crop_Damage -0.016494705652070066
Correlation withSoil_Type Crop_Damage -0.021997272041923074
Correlation withPesticide_Use_Category Crop_Damage 0.17500204470503275
Correlation withNumber_Doses_Week Crop_Damage -0.03087425832208959
Correlation withNumber_Weeks_Used Crop_Damage 0.1746679539287501
Correlation withNumber_Weeks_Quit Crop_Damage -0.13311659360152783


In [174]:
df.agg({"Number_Weeks_Used":"mean"}).collect()

[Row(avg(Number_Weeks_Used)=28.62397004683313)]

In [95]:
from pyspark.sql.types import *

In [175]:
def convertColumn(df_string, names, newType):
    for name in names: 
        df_string = df_string.withColumn(name, df_string[name].cast(newType))
    return df_string

In [176]:
categorical_feautres=['Crop_Type','Soil_Type','Pesticide_Use_Category','Number_Doses_Week','Season']
df= convertColumn(df, categorical_feautres, FloatType())

In [177]:
df.printSchema()


root
 |-- Estimated_Insects_Count: integer (nullable = true)
 |-- Crop_Type: float (nullable = true)
 |-- Soil_Type: float (nullable = true)
 |-- Pesticide_Use_Category: float (nullable = true)
 |-- Number_Doses_Week: float (nullable = true)
 |-- Number_Weeks_Used: integer (nullable = true)
 |-- Number_Weeks_Quit: integer (nullable = true)
 |-- Season: float (nullable = true)
 |-- Crop_Damage: integer (nullable = true)



In [178]:
from pyspark.ml.feature import StringIndexer

In [179]:
df.groupBy("Crop_Type").count().sort("count",ascending=True).show()	

+---------+-----+
|Crop_Type|count|
+---------+-----+
|      1.0|25269|
|      0.0|63589|
+---------+-----+



In [180]:
df.groupBy("Crop_Damage").count().sort("count",ascending=True).show()	

+-----------+-----+
|Crop_Damage|count|
+-----------+-----+
|          2| 2313|
|          1|12307|
|          0|74238|
+-----------+-----+



In [102]:
df.describe().show()

+-------+-----------------------+------------------+-------------------+----------------------+------------------+------------------+-----------------+------------------+-------------------+
|summary|Estimated_Insects_Count|         Crop_Type|          Soil_Type|Pesticide_Use_Category| Number_Doses_Week| Number_Weeks_Used|Number_Weeks_Quit|            Season|        Crop_Damage|
+-------+-----------------------+------------------+-------------------+----------------------+------------------+------------------+-----------------+------------------+-------------------+
|  count|                  88858|             88858|              88858|                 88858|             88858|             79858|            88858|             88858|              88858|
|   mean|     1399.0122104931463|0.2843750703369421| 0.4584167998379437|    2.2641855544801817|25.849951608183844| 28.62397004683313|9.589986270228904|1.8969591933196785|0.19056247045848432|
| stddev|      849.0487806668847|0.4511188091

In [181]:
df.crosstab('Number_Doses_Week', 'Crop_Damage').show()

+-----------------------------+-----+----+---+
|Number_Doses_Week_Crop_Damage|    0|   1|  2|
+-----------------------------+-----+----+---+
|                          0.0|  118| 554|165|
|                          5.0| 5007| 473| 40|
|                         10.0|10422|1542|147|
|                         25.0| 4310| 468|112|
|                         20.0|17789|4042|758|
|                         60.0| 3836| 435| 96|
|                         85.0|    9|   1|  1|
|                         70.0|  275|  21|  7|
|                         65.0|   57|   7|  0|
|                         45.0|  868|  55| 13|
|                         80.0|  590|  68| 14|
|                         35.0| 1647| 155| 30|
|                         95.0|   28|   2|  0|
|                         50.0| 2995| 300| 66|
|                         40.0|10092|1797|409|
|                         55.0|  250|  25|  2|
|                         75.0|   56|   5|  0|
|                         30.0| 8708|1399|331|
|            

In [182]:
df.groupby('Crop_Damage').agg({'Number_Doses_Week': 'mean'}).show()	

+-----------+----------------------+
|Crop_Damage|avg(Number_Doses_Week)|
+-----------+----------------------+
|          1|    24.176078654424312|
|          2|     26.11327280587981|
|          0|    26.119238126027103|
+-----------+----------------------+



In [105]:
import  pyspark.sql.functions as F

In [106]:
df.columns

['Estimated_Insects_Count',
 'Crop_Type',
 'Soil_Type',
 'Pesticide_Use_Category',
 'Number_Doses_Week',
 'Number_Weeks_Used',
 'Number_Weeks_Quit',
 'Season',
 'Crop_Damage']

In [107]:
from pyspark.sql.functions import udf,col
from pyspark.sql.functions import col,sum,avg,max

In [183]:
df.agg({"Number_Weeks_Used":"mean"}).collect()

[Row(avg(Number_Weeks_Used)=28.62397004683313)]

In [184]:
df=df.na.fill(value=28.62397004683313,subset=["Number_Weeks_Used"])

In [185]:
from pyspark.sql.functions import approx_count_distinct,collect_list
from pyspark.sql.functions import collect_set,sum,avg,max,countDistinct,count
from pyspark.sql.functions import first, last, kurtosis, min, mean, skewness 
from pyspark.sql.functions import stddev, stddev_samp, stddev_pop, sumDistinct
from pyspark.sql.functions import variance,var_samp,  var_pop

In [186]:
df.select(kurtosis("Estimated_Insects_Count")).show(truncate=False)
df.select(max("Estimated_Insects_Count")).show(truncate=False)
df.select(min("Estimated_Insects_Count")).show(truncate=False)
df.select(mean("Estimated_Insects_Count")).show(truncate=False)
df.select(skewness("Estimated_Insects_Count")).show(truncate=False)
df.select(stddev("Estimated_Insects_Count"), stddev_samp("Estimated_Insects_Count"),stddev_pop("Estimated_Insects_Count")).show(truncate=False)

+---------------------------------+
|kurtosis(Estimated_Insects_Count)|
+---------------------------------+
|0.14001746588196706              |
+---------------------------------+

+----------------------------+
|max(Estimated_Insects_Count)|
+----------------------------+
|4097                        |
+----------------------------+

+----------------------------+
|min(Estimated_Insects_Count)|
+----------------------------+
|150                         |
+----------------------------+

+----------------------------+
|avg(Estimated_Insects_Count)|
+----------------------------+
|1399.0122104931463          |
+----------------------------+

+---------------------------------+
|skewness(Estimated_Insects_Count)|
+---------------------------------+
|0.8391314724409544               |
+---------------------------------+

+------------------------------------+------------------------------------+-----------------------------------+
|stddev_samp(Estimated_Insects_Count)|stddev_samp(Estimate

In [217]:
cols=df.columns

In [213]:
from pyspark.ml.feature import OneHotEncoder
stages = []

In [214]:
for categoricalCol in categorical_feautres:
  encoder = OneHotEncoder(inputCols=[categoricalCol],outputCols=[categoricalCol + "classVec"])
  stages += [ encoder]

In [215]:
from pyspark.ml.feature import  StringIndexer, VectorAssembler

In [216]:
label_stringIdx = StringIndexer(inputCol = 'Crop_Damage', outputCol = 'label')
stages += [label_stringIdx]

assemblerInputs = categorical_feautres + numeric_feautres
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [218]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df)
df = pipelineModel.transform(df)
selectedCols = ['label', 'features'] + cols
df = df.select(selectedCols)
df.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- Estimated_Insects_Count: integer (nullable = true)
 |-- Crop_Type: float (nullable = true)
 |-- Soil_Type: float (nullable = true)
 |-- Pesticide_Use_Category: float (nullable = true)
 |-- Number_Doses_Week: float (nullable = true)
 |-- Number_Weeks_Used: integer (nullable = true)
 |-- Number_Weeks_Quit: integer (nullable = true)
 |-- Season: float (nullable = true)
 |-- Crop_Damage: integer (nullable = true)



In [219]:
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
label,0,1,1,1,1
features,"(1.0, 0.0, 1.0, 0.0, 1.0, 188.0, 0.0, 0.0)","(1.0, 0.0, 1.0, 0.0, 2.0, 209.0, 0.0, 0.0)","(1.0, 0.0, 1.0, 0.0, 2.0, 257.0, 0.0, 0.0)","[1.0, 1.0, 1.0, 0.0, 2.0, 257.0, 0.0, 0.0]","(1.0, 0.0, 1.0, 0.0, 2.0, 342.0, 0.0, 0.0)"
Estimated_Insects_Count,188,209,257,257,342
Crop_Type,1,1,1,1,1
Soil_Type,0,0,0,1,0
Pesticide_Use_Category,1,1,1,1,1
Number_Doses_Week,0,0,0,0,0
Number_Weeks_Used,0,0,0,0,0
Number_Weeks_Quit,0,0,0,0,0
Season,1,2,2,2,2


In [220]:
train, test = df.randomSplit([0.7, 0.3], seed = 2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 62220
Test Dataset Count: 26638


In [221]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)

linearModel = lr.fit(train)

In [238]:
print("Coefficients: " + str(linearModel.coefficientMatrix))
print("Intercept: " + str(linearModel.interceptVector))

Coefficients: DenseMatrix([[ 8.31616905e-02,  4.46522192e-02, -1.56510957e-01,
               1.94372345e-03,  2.13775652e-02, -1.68395487e-04,
              -5.30965470e-03,  1.13246911e-02],
             [-7.56020827e-02, -1.56278759e-02,  1.07980910e-01,
              -1.69270842e-03,  1.67008572e-03,  1.45141745e-04,
               1.81917009e-03, -3.66417511e-03],
             [-7.55960788e-03, -2.90243432e-02,  4.85300469e-02,
              -2.51015028e-04, -2.30476509e-02,  2.32537417e-05,
               3.49048461e-03, -7.66051594e-03]])
Intercept: [2.273137064002961,-0.4105979919208667,-1.8625390720820942]


In [241]:
predictions = linearModel.transform(test)
predictions.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- Estimated_Insects_Count: integer (nullable = true)
 |-- Crop_Type: float (nullable = true)
 |-- Soil_Type: float (nullable = true)
 |-- Pesticide_Use_Category: float (nullable = true)
 |-- Number_Doses_Week: float (nullable = true)
 |-- Number_Weeks_Used: integer (nullable = true)
 |-- Number_Weeks_Quit: integer (nullable = true)
 |-- Season: float (nullable = true)
 |-- Crop_Damage: integer (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [242]:
selected = predictions.select("label", "prediction", "probability")
selected.show(20)

+-----+----------+--------------------+
|label|prediction|         probability|
+-----+----------+--------------------+
|  0.0|       0.0|[0.91179125761333...|
|  0.0|       0.0|[0.87319801604140...|
|  0.0|       0.0|[0.89148547400262...|
|  0.0|       0.0|[0.88930866619271...|
|  0.0|       0.0|[0.85656638696019...|
|  0.0|       0.0|[0.83541858937701...|
|  0.0|       0.0|[0.91343141109526...|
|  0.0|       0.0|[0.90683693511043...|
|  0.0|       0.0|[0.90511431769095...|
|  0.0|       0.0|[0.89798068728243...|
|  0.0|       0.0|[0.89243982463707...|
|  0.0|       0.0|[0.87541535947928...|
|  0.0|       0.0|[0.86257022182329...|
|  0.0|       0.0|[0.84559795854296...|
|  0.0|       0.0|[0.76834596441293...|
|  0.0|       0.0|[0.88632863908758...|
|  0.0|       0.0|[0.87126406077529...|
|  0.0|       0.0|[0.84217706014747...|
|  0.0|       0.0|[0.82034571057543...|
|  0.0|       0.0|[0.86495175680557...|
+-----+----------+--------------------+
only showing top 20 rows



In [245]:
def accuracy_m(model): 
    predictions = model.transform(test)
    cm = predictions.select("label", "prediction")
    acc = cm.filter(cm.label == cm.prediction).count() / cm.count()
    print("Model accuracy: %.3f%%" % (acc * 100)) 
accuracy_m(model = linearModel)


Model accuracy: 83.569%
